## 6 - Neural Scaling - MNIST example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import umap #pip install umap-learn
from torch.optim.lr_scheduler import StepLR

In [ ]:
umap.UMAP

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 16) #128)
        self.fc2 = nn.Linear(16, 10) #(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

    def get_penultimate_out(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

    def get_penulatimate_penultimate_out(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        return x

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    losses=[]
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            losses.append(loss.item())
    return losses

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,batch_size=64)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1000)

In [5]:
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=0.05)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
device='cpu'

In [6]:
fc1_outs=[]
losses=[]
epoch=1
viz_period=100

for batch_idx, (data, target) in enumerate(train_loader):
    model.train()
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % viz_period == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                fc1_out=model.get_penultimate_out(data)
                # fc1_out=model.get_penulatimate_penultimate_out(data)
                break
        fc1_outs.append(fc1_out.numpy())

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.323201
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.417735
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.277343
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.168514
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.258615
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.114716
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.065391
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.002706
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.180183
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.787895


In [7]:
u=umap.UMAP(n_components=2, verbose=True, n_neighbors=20, min_dist=0.01, metric="cosine", random_state=0)
layout=u.fit_transform(fc1_out.numpy())
layout = normalize_layout(layout)

AttributeError: module 'umap' has no attribute 'UMAP'

In [ ]:
fig=plt.figure(0, (6,6), facecolor='w') 
ax=fig.add_subplot(111, facecolor='w') 
scatter(layout[:,0], layout[:,1], c=target.numpy(), alpha=1.0, s=30, cmap='tab10', linewidth=0)
plt.grid(1)

legend_colors = plt.cm.tab10(np.arange(10))
legend_elements = [Line2D([0], [0], marker='o', color='#F0E5D1', alpha=1.0, markerfacecolor=legend_colors[i], markersize=10, label=str(i))
                   for i in range(10)]
plt.legend(handles=legend_elements, facecolor='#F0E5D1', loc='lower left', bbox_to_anchor=(1, 0.5))